<h1 align=center><font size = 5><em></em>Part1 of Segmenting and Clustering Neighborhoods in Toronto by Haowen Wang</font></h1>

## Part 1


In [4]:
import requests # library to handle requests
!conda install -c conda-forge lxml==3.7.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import lxml.html as lh
import lxml
import urllib.request
import numpy as np # library to handle data in a vectorized manner
import pandas as pd # library for data analsysis

!conda install -c conda-forge beautifulsoup4 --yes # uncomment this line if you haven't completed the Foursquare API lab
from bs4 import BeautifulSoup

print('Libraries imported.')

Solving environment: done


==> WARNING: A newer version of conda exists. <==
  current version: 4.5.11
  latest version: 4.8.1

Please update conda by running

    $ conda update -n base -c defaults conda



# All requested packages already installed.

Solving environment: done


==> WARNING: A newer version of conda exists. <==
  current version: 4.5.11
  latest version: 4.8.1

Please update conda by running

    $ conda update -n base -c defaults conda



# All requested packages already installed.

Libraries imported.


In [5]:
url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'

#### Using bs4 to scrape

In [99]:
def scrape_data(cname = 'wikitable sortable', cols = 3):
    page = urllib.request.urlopen(url).read()
    soup = BeautifulSoup(page,'html.parser')
    table = soup.find('table', class_ = cname)
    header = [head.findAll(text = True)[0].strip() for head in table.find_all('th')]
    data = [[da.findAll(text = True)[0].strip() for da in dat.find_all('td')] for dat in table.find_all('tr')]
    data = [row for row in data if len(row) == cols]
    data = pd.DataFrame(data, columns = header)
    return data
data = scrape_data()
data = data[~data['Borough'].str.contains('Not assigned')]

data[data['Neighbourhood']=='Not assigned']['Neighbourhood'] = data[data['Neighbourhood']=='Not assigned']['Borough']



/home/jupyterlab/conda/envs/python/lib/python3.6/site-packages/ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  del sys.path[0]


In [115]:
# data.columns[' '] = []
Nei = []
Bor = []
Pos = []
for i in data['Postcode'].unique():
    temp = data[data['Postcode']==i]
    Pos.append(i)
    Nei.append(list(temp['Neighbourhood'].unique()))
    Bor.append(list(temp['Borough'].unique()))

# verify that if neibourhood has 'Not assigned'
print('If ‘Not assigned’ in the Neighbourhood?:','Not assigned' in Nei)       

If ‘Not assigned’ in the Neighbourhood?: False


In [127]:
temp = data.groupby('Postcode')
data_Toronto = pd.DataFrame({'Postcode':Pos, 'Borough':Bor, 'Neighbourhood':Nei})
data_Toronto.sort_values('Postcode', inplace = True)
data_Toronto.reset_index(inplace = True)
data_Toronto = data_Toronto.iloc[:,1:]
data_Toronto

,Postcode,Borough,Neighbourhood
0,M1B,[Scarborough],"[Rouge, Malvern]"
1,M1C,[Scarborough],"[Highland Creek, Rouge Hill, Port Union]"
2,M1E,[Scarborough],"[Guildwood, Morningside, West Hill]"
3,M1G,[Scarborough],[Woburn]
4,M1H,[Scarborough],[Cedarbrae]
...,...,...,...
98,M9N,[York],[Weston]
99,M9P,[Etobicoke],[Westmount]
100,M9R,[Etobicoke],"[Kingsview Village, Martin Grove Gardens, Rich..."
101,M9V,[Etobicoke],"[Albion Gardens, Beaumond Heights, Humbergate,..."


In [128]:
data_Toronto.to_csv('data.csv', index = False)

In [129]:
data_Toronto.shape

(103, 3)